In [1]:
import os
import platform
import networkx as nx
from graph_partitioning import GraphPartitioning, utils

pwd = %pwd

config = {

    "DATA_FILENAME": os.path.join(pwd, "data", "oneshot_fennel_weights.txt"),
    "OUTPUT_DIRECTORY": os.path.join(pwd, "output"),

    # Set which algorithm is run for the PREDICTION MODEL.
    # Either: 'FENNEL' or 'SCOTCH'
    "PREDICTION_MODEL_ALGORITHM": "PATOH",

    # Alternativly, read input file for prediction model.
    # Set to empty to generate prediction model using algorithm value above.
    "PREDICTION_MODEL": "",

    
    "PARTITIONER_ALGORITHM": "PATOH",

    # File containing simulated arrivals. This is used in simulating nodes
    # arriving at the shelter. Nodes represented by line number; value of
    # 1 represents a node as arrived; value of 0 represents the node as not
    # arrived or needing a shelter.
    "SIMULATED_ARRIVAL_FILE": os.path.join(pwd, "data", "simulated_arrival.txt"),

    # File containing the geographic location of each node.
    "POPULATION_LOCATION_FILE": os.path.join(pwd, "data", "population_location.csv"),

    # Number of shelters
    "num_partitions": 4,

    # The number of iterations when making prediction model
    "num_iterations": 10,

    # Percentage of prediction model to use before discarding
    # When set to 0, prediction model is discarded, useful for one-shot
    "prediction_model_cut_off": 0.10,

    # Alpha value used in one-shot (when restream_batches set to 1)
    "one_shot_alpha": 0.5,

    # Number of arrivals to batch before recalculating alpha and restreaming.
    # When set to 1, one-shot is used with alpha value from above
    "restream_batches": 100,

    # When the batch size is reached: if set to True, each node is assigned
    # individually as first in first out. If set to False, the entire batch
    # is processed and empty before working on the next batch.
    "sliding_window": False,

    # Create virtual nodes based on prediction model
    "use_virtual_nodes": False,

    # Virtual nodes: edge weight
    "virtual_edge_weight": 1.0,


    ####
    # GRAPH MODIFICATION FUNCTIONS

    # Also enables the edge calculation function.
    "graph_modification_functions": True,

    # If set, the node weight is set to 100 if the node arrives at the shelter,
    # otherwise the node is removed from the graph.
    "alter_arrived_node_weight_to_100": True,

    # Uses generalized additive models from R to generate prediction of nodes not
    # arrived. This sets the node weight on unarrived nodes the the prediction
    # given by a GAM.
    # Needs POPULATION_LOCATION_FILE to be set.
    "alter_node_weight_to_gam_prediction": True,

    # The value of 'k' used in the GAM will be the number of nodes arrived until
    # it reaches this max value.
    "gam_k_value": 100,

    # Alter the edge weight for nodes that haven't arrived. This is a way to
    # de-emphasise the prediction model for the unknown nodes.
    "prediction_model_emphasis": 1.0,
    
    
    "SCOTCH_LIB_PATH": os.path.join(pwd, "libs/scotch/macOS/libscotch.dylib")
    if 'Darwin' in platform.system()
    else "/usr/local/lib/libscotch.so",
    
    # Path to the PaToH shared library
    "PATOH_LIB_PATH": os.path.join(pwd, "libs/patoh/lib/macOS/libpatoh.dylib")
    if 'Darwin' in platform.system()
    else os.path.join(pwd, "libs/patoh/lib/linux/libpatoh.so"),

    "SCOTCH_PYLIB_REL_PATH": os.path.join(pwd, "..", "csap-graphpartitioning", "src", "python")
}

gp = GraphPartitioning(config)

# Optional: shuffle the order of nodes arriving
# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(gp.arrival_order)

%pylab inline

import pandas as pd
cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "MODULARITY"]

Populating the interactive namespace from numpy and matplotlib


In [2]:
gp.load_network()

Graph loaded...
Name: 
Type: Graph
Number of nodes: 1000
Number of edges: 2939
Average degree:   5.8780
Graph is undirected


In [3]:
gp.init_partitioner()

PaToH partitioner loaded for generating PREDICTION MODEL.
PaToH partitioner loaded for making shelter assignments.


# Prediction Model

In [4]:
m = gp.prediction_model()

AttributeError: 'PATOHParameters' object has no attribute 'params'

In [ ]:
rows = list(range(1, len(m)+1))
df = pd.DataFrame(m, index=rows, columns=cols)
print(df)

part = dict(zip(gp.G.nodes(), gp.assignments))
values = [part.get(node) for node in gp.G.nodes()]

pos = nx.spring_layout(gp.G)
plt.figure(figsize=(10, 10))
plt.axis('off')
colours = {0: 'red', 1: 'blue', 2: 'yellow', 3: 'green'}

colour = []
for n in gp.G.nodes_iter():
    colour += [colours[gp.assignments[n]]]

nx.draw_networkx_nodes(gp.G, pos, node_size=20, node_color=colour, cmap=plt.cm.jet, with_labels=False)
nx.draw_networkx_edges(gp.G, pos, alpha=0.2)
plt.show(gp.G)


# Assign Cut Off

In [ ]:
gp.assign_cut_off()

# Run Simulation

In [ ]:
m = gp.batch_arrival()

In [ ]:
rows = list(range(1, len(m)+1))
df = pd.DataFrame(m, index=rows, columns=cols)

df.plot()
print(df)

# Network Visual

In [ ]:
def draw_partition(graph, partition_nodes, fixed_positions):
    fixed_nodes = fixed_positions.keys()
    pos = nx.spring_layout(graph, k=1, iterations=10, pos=fixed_positions, fixed=fixed_nodes)
    plt.figure(figsize=(8, 8))
    plt.axis('off')

    size = [] 
    colour = []
    labels = {}
    for n in graph.nodes_iter():
        if n in partition_nodes:
            size += [100]
            colour += ['b']
            labels[n] = 'Partition ' + str(partition_nodes.index(n))
        else:
            size += [30]
            colour += ['r']

    nx.draw_networkx_nodes(graph, pos, node_size=size, node_color=colour, with_labels=False)
    nx.draw_networkx_edges(graph, pos, alpha=0.9)
    nx.draw_networkx_labels(graph, pos, labels, font_size=16, font_family='sans-serif')
    plt.show(graph)

partition_nodes = list(range(1000, 1000 + gp.num_partitions))

print('Partition 0')
fixed_positions = {1001: (0, 1), 1002: (-1, 0), 1003: (1, 0)}
part_graph = utils.squash_partition(gp.G, partition_nodes, 0, gp.assignments)
draw_partition(part_graph, partition_nodes, fixed_positions)

print('Partition 1')
fixed_positions = {1000: (0, 1), 1002: (-1, 0), 1003: (1, 0)}
part_graph = utils.squash_partition(gp.G, partition_nodes, 1, gp.assignments)
draw_partition(part_graph, partition_nodes, fixed_positions)

print('Partition 2')
fixed_positions = {1000: (0, 1), 1001: (-1, 0), 1003: (1, 0)}
part_graph = utils.squash_partition(gp.G, partition_nodes, 2, gp.assignments)
draw_partition(part_graph, partition_nodes, fixed_positions)

print('Partition 3')
fixed_positions = {1000: (0, 1), 1001: (-1, 0), 1002: (1, 0)}
part_graph = utils.squash_partition(gp.G, partition_nodes, 3, gp.assignments)
draw_partition(part_graph, partition_nodes, fixed_positions)

In [ ]:
part = dict(zip(gp.G.nodes(), gp.assignments))
values = [part.get(node) for node in gp.G.nodes()]

pos = nx.spring_layout(gp.G, k=0.5, iterations=20)
plt.figure(figsize=(10, 10))
plt.axis('off')
colours = {0: 'red', 1: 'blue', 2: 'yellow', 3: 'green'}

colour = []
for n in gp.G.nodes_iter():
    colour += [colours[gp.assignments[n]]]

nx.draw_networkx_nodes(gp.G, pos, node_size=20, node_color=colour, cmap=plt.cm.jet, with_labels=False)
nx.draw_networkx_edges(gp.G, pos, alpha=0.5)
plt.show(gp.G)

In [ ]:
gp.get_metrics()